In [1]:
# importing libraries.
import numpy as np 
import pandas as pd 
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_rows', 100)
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [2]:
# Reading train dataset.
train = pd.read_csv('train_v2.csv',index_col='id')
train.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f770,f771,f772,f773,f774,f775,f776,f777,f778,loss
id,,,,,,,,,,,,,,,,,,,,,
1,126,10,0.69,1100,3,13699,7201.00,4949.00,126.75,126.03,...,5,2.14,-1.54,1.18,0.18,0.79,1,0,5,0
2,121,10,0.78,1100,3,84645,240.00,1625.00,123.52,121.35,...,6,0.54,-0.24,0.13,0.19,-0.68,1,0,5,0
3,126,10,0.50,1100,3,83607,1800.00,1527.00,127.76,126.49,...,13,2.89,-1.73,1.04,0.25,0.73,1,0,5,0
4,134,10,0.44,1100,3,82642,7542.00,1730.00,132.94,133.58,...,4,1.29,-0.89,0.66,0.25,0.71,1,0,5,0
5,109,9,0.50,2900,4,79124,89.00,491.00,122.72,112.77,...,26,6.11,-3.82,2.51,0.23,-0.54,0,0,5,0


In [3]:
# Showing information about train data.
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 105471 entries, 1 to 105471
Columns: 770 entries, f1 to loss
dtypes: float64(653), int64(98), object(19)
memory usage: 620.4+ MB


In [4]:
# Reading test data.
test = pd.read_csv('test_v2.csv',index_col='id')
test.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f769,f770,f771,f772,f773,f774,f775,f776,f777,f778
id,,,,,,,,,,,,,,,,,,,,,
105472,147,6,0.49,1100,17.00,75506,964.00,12686.00,152.63,115.91,...,-8.71,19,3.30,-9.37,0.50,0.05,-1.07,0,1,1079
105473,156,6,0.73,5400,3.00,79754,1455.00,4803.00,153.95,155.50,...,-13.26,24,9.53,-7.55,6.22,0.30,0.61,0,1,36
105474,132,9,0.90,2200,16.00,113,5735.00,2387.00,130.61,131.96,...,-4.99,9,3.25,-2.33,1.69,0.23,0.42,0,0,393
105475,128,7,0.04,1300,4.00,3793,4689.00,3469.00,120.50,121.93,...,-16.83,11,0.26,-5.31,0.78,0.28,-0.77,0,0,394
105476,119,10,0.44,1300,16.00,13026,2788.00,7438.00,127.00,125.98,...,-20.00,10,13.55,-0.61,0.01,0.18,-1.08,0,0,23


In [5]:
# Showing information about test data.
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 210944 entries, 105472 to 316415
Columns: 769 entries, f1 to f778
dtypes: float64(654), int64(95), object(19), uint64(1)
memory usage: 1.2+ GB


In [6]:
# Creating a function that shows how many null values in the data.
def df_nulls(df):
    isnull = df.isnull().sum().sort_values(ascending=False).to_frame() 
    isnull.columns = ['How_many']
    isnull['precentage'] = np.around(((isnull / len(df) * 100)[(isnull / len(df) * 100) != 0]), decimals=2)
    print (f"Your selected dataframe has  {str(df.shape[1])} columns.\n There are  {str(len(isnull[isnull.How_many>0]))} columns that have missing values.")
    return isnull

In [7]:
# how many null values in train data.
df_nulls(train).head(20)

Your selected dataframe has  770 columns.
 There are  525 columns that have missing values.


,How_many,precentage
f662,18833,17.86
f663,18833,17.86
f159,18736,17.76
f160,18736,17.76
f169,18417,17.46
f170,18417,17.46
f618,18407,17.45
f619,18407,17.45
f331,18067,17.13
f330,18067,17.13


In [8]:
# how many null values in test data.
df_nulls(test).head(20)

Your selected dataframe has  769 columns.
 There are  530 columns that have missing values.


,How_many,precentage
f663,36514,17.31
f662,36514,17.31
f159,35818,16.98
f160,35818,16.98
f619,34030,16.13
f618,34030,16.13
f170,33764,16.01
f169,33764,16.01
f331,33594,15.93
f330,33594,15.93


In [9]:
# Geting the names of the numrical columns.
num_columns = [column for column in train.columns if train[column].dtype != 'object']
num_columns

['f1',
 'f2',
 'f3',
 'f4',
 'f5',
 'f6',
 'f7',
 'f8',
 'f9',
 'f10',
 'f13',
 'f14',
 'f15',
 'f16',
 'f17',
 'f18',
 'f19',
 'f20',
 'f21',
 'f22',
 'f23',
 'f24',
 'f25',
 'f26',
 'f27',
 'f28',
 'f29',
 'f30',
 'f31',
 'f32',
 'f33',
 'f34',
 'f35',
 'f36',
 'f37',
 'f38',
 'f39',
 'f40',
 'f41',
 'f42',
 'f43',
 'f44',
 'f45',
 'f46',
 'f47',
 'f48',
 'f49',
 'f50',
 'f51',
 'f52',
 'f53',
 'f54',
 'f55',
 'f56',
 'f57',
 'f58',
 'f59',
 'f60',
 'f61',
 'f62',
 'f63',
 'f64',
 'f65',
 'f66',
 'f67',
 'f68',
 'f69',
 'f70',
 'f71',
 'f72',
 'f73',
 'f74',
 'f75',
 'f76',
 'f77',
 'f78',
 'f79',
 'f80',
 'f81',
 'f82',
 'f83',
 'f84',
 'f85',
 'f86',
 'f87',
 'f88',
 'f89',
 'f90',
 'f91',
 'f92',
 'f93',
 'f94',
 'f95',
 'f96',
 'f97',
 'f98',
 'f99',
 'f100',
 'f101',
 'f102',
 'f103',
 'f104',
 'f105',
 'f106',
 'f107',
 'f108',
 'f109',
 'f110',
 'f111',
 'f112',
 'f113',
 'f114',
 'f115',
 'f116',
 'f117',
 'f118',
 'f119',
 'f120',
 'f121',
 'f122',
 'f123',
 'f124',
 'f125',

In [10]:
test = test[num_columns[:-1]] # chosing only the numric columns from test data (after excluding loss data).
train = train[num_columns] # chosing only the numric columns from train data 

In [11]:
train.dropna(axis=0,subset = ['loss'], inplace = True) # droping row with null values in loss column.

X_train_full = train.drop(['loss'], axis =1) # droping loss column to create X train data.
y_train_full = train.loss # giving loss column to y as the target data.

X_train, X_valid, y_train, y_valid = train_test_split(X_train_full,y_train_full,test_size=0.2, random_state=42) # spliting the data to train and validation data.

In [12]:
# A numrical transformer.
num_trans = Pipeline(steps = [
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler',StandardScaler())
])

num_columns.remove('loss')

# A preprocessor that combines the two previous transformers.
preprocessor = ColumnTransformer(transformers = [
    ('num', num_trans, num_columns)
],
    remainder = "drop")

In [13]:
X_train_trans = preprocessor.fit_transform(X_train) # Preprocessing train data.
X_valid_trans = preprocessor.transform(X_valid) # Preprocessing validation data.

In [14]:
X_train_trans.shape # chincking the shape of the preprocessed data.

(84376, 750)

In [15]:
# creating a function that transform the preprocessed data that is in numpy array form to DataFrame form.
def array_to_df(arr):
    return pd.DataFrame(data = arr,columns = X_train.columns)

In [16]:
X_train_trans = array_to_df(X_train_trans)
X_valid_trans = array_to_df(X_valid_trans)

In [17]:
# creating a LogisticRegression model that will help us choosing the right features.
from sklearn.linear_model import LogisticRegression

my_model = LogisticRegression().fit(X_train_trans, y_train)

In [19]:
# Geting the most effictive features on predictions.
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(my_model, random_state=1).fit(X_valid_trans.head(1000), y_valid.head(1000))
eli5.show_weights(perm, feature_names = X_valid_trans.columns.tolist())

Weight,Feature
0.0128 ± 0.0015,f596
0.0066 ± 0.0027,f2
0.0034 ± 0.0010,f664
0.0028 ± 0.0008,f589
0.0024 ± 0.0024,f524
0.0024 ± 0.0010,f361
0.0020 ± 0.0018,f674
0.0020 ± 0.0000,f636
0.0018 ± 0.0008,f269
0.0018 ± 0.0008,f634


In [20]:
# Choosing inly the important features
from sklearn.feature_selection import SelectFromModel

sel = SelectFromModel(perm, threshold=0.001, prefit=True)
X_train_super_trans = sel.transform(X_train_trans)
X_valid_super_trans = sel.transform(X_valid_trans)

In [21]:
# checking the shape of the filtered data.
X_train_super_trans.shape

(84376, 47)

In [22]:
train.shape

(105471, 751)

In [23]:
# Creating functions to get the cross validation score.
from sklearn.model_selection import cross_val_score

def cross_val(X_train_super_trans, y_train, model):
    # Applying k-Fold Cross Validation
    accuracies = cross_val_score(estimator = model, X = X_train_super_trans, y = y_train, cv = 5)
    return accuracies.mean()

# Takes in a model, trains the model, and evaluates the model on the test set
def fit_and_evaluate(model):
    
    # Train the model
    model.fit(X_train_super_trans, y_train)
    
    # evalute
    model_cross = cross_val(X_train_super_trans, y_train, model)
    
    # Return the performance metric
    return model_cross

In [24]:
# # Random Forest Classification
from sklearn.ensemble import RandomForestClassifier
random = RandomForestClassifier(n_estimators = 10, criterion = 'entropy')
random_cross = fit_and_evaluate(random)

print('Random Forest Performance on the test set: Cross Validation Score = %0.4f' % random_cross)

Random Forest Performance on the test set: Cross Validation Score = 0.9074


In [25]:
# Preprocessing and filtering the whole train and test data after checking its Cross Validation Score.
new_train = preprocessor.fit_transform(X_train_full)
new_test = preprocessor.transform(test)

new_train = sel.transform(new_train)
new_test = sel.transform(new_test)

In [27]:
# reading the sample submission dataset
submission = pd.read_csv('sampleSubmission.csv')
submission.head()

,id,loss
0,105472,0
1,105473,0
2,105474,0
3,105475,0
4,105476,0


In [28]:
# fitting the RandomForestClassifier with the whole train data to make it's predictions more accurate.
random.fit(new_train,y_train_full)

submission.loss = random.predict(new_test)

submission.head(40)

,id,loss
0,105472,0
1,105473,0
2,105474,0
3,105475,0
4,105476,0
5,105477,0
6,105478,0
7,105479,0
8,105480,0
9,105481,0


In [29]:
 submission.loss.unique()

array([ 0,  2,  1,  4,  3, 11,  8,  6,  7], dtype=int64)

In [30]:
# Making submission.
submission.to_csv('submission.csv', index=False, header=True)